# Photon Sensitivity Evaluation

The following notebook downloads two-photon data sets from the DANDI Archive and generates figures depicting estimations of the Photon Transfer Function and photon flux images. 

In [ ]:
%matplotlib inline
from pynwb import NWBHDF5IO
import numpy as np 
import matplotlib.pyplot as plt
from dandi.dandiapi import DandiAPIClient
from compress_multiphoton import compute_sensitivity
from fsspec.implementations.cached import CachingFileSystem
import pathlib
import fsspec
import pynwb
import h5py
import colorcet as cc

In [ ]:
def make_figure(scan, figure_filename, title=None):
    
    qs = compute_sensitivity(scan.transpose(1,2,0))
    print('Quantal size: {sensitivity}\nIntercept: {zero_level}\n'.format(**qs))
    
    fig, axx = plt.subplots(2,2, figsize=(8, 12), tight_layout=True)
    q = qs['sensitivity']
    b = qs['zero_level']
    axx = iter(axx.flatten())

    ax = next(axx)
    m = scan.mean(axis=0)
    _ = ax.imshow(m, vmin=0, vmax=np.quantile(m, 0.999), cmap='gray')
    ax.axis(False)
    cbar = plt.colorbar(_, ax=ax, ticks=[0.05, .5, 0.95])
    cbar.remove()
    ax.set_title('average')
    label = "A"
    ax.text(-0.1, 1.15, label, transform=ax.transAxes,
      fontsize=14, fontweight='bold', va='top', ha='right')

    ax = next(axx)
    x = np.arange(qs["min_intensity"], qs["max_intensity"])
    fit = qs["model"].predict(x.reshape(-1, 1))
    ax.scatter(x, np.minimum(fit[-1]*2, qs["variance"]), s=2, alpha=0.5)
    ax.plot(x, fit, 'r')
    ax.grid(True)
    ax.set
    ax.set_xlabel('intensity')
    ax.set_ylabel('variance')
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.set_title('Photon Transfer Curve')
    label = "B"
    ax.text(-0.1, 1.15, label, transform=ax.transAxes,
      fontsize=14, fontweight='bold', va='top', ha='right')

    ax = next(axx)
    v = ((scan[1:,:,:].astype('float64') - scan[:-1,:,:]) ** 2/2).mean(axis=0)
    imx = np.stack(((m-b)/q, v/q/q, (m-b)/q), axis=-1)
    _ = ax.imshow(np.minimum(1, np.sqrt(0.01 + np.maximum(0, imx/np.quantile(imx, 0.9999))) - 0.1), cmap='PiYG')
    cbar = plt.colorbar(_, ax=ax, ticks=[0.2, .5, 0.8])
    cbar.ax.set_yticklabels(['<< 1', '1', '>> 1'])  
    ax.axis(False)
    ax.set_title('coefficient of variation')
    label = "C"
    ax.text(-0.1, 1.15, label, transform=ax.transAxes,
      fontsize=14, fontweight='bold', va='top', ha='right')

    ax = next(axx)
    im = (scan.mean(axis=0)-qs['zero_level'])/qs['sensitivity']
    mx = np.quantile(im, 0.999)
    _ = ax.imshow(im, vmin=-mx, vmax=mx, cmap=cc.cm.CET_D13)
    plt.colorbar(_, ax=ax)
    ax.axis(False)
    ax.set_title('Quantum flux\nphotons / pixel / frame');
    label = "D"
    ax.text(-0.1, 1.15, label, transform=ax.transAxes,
      fontsize=14, fontweight='bold', va='top', ha='right')

    plt.suptitle(f'{title or figure_filename}\nPhoton sensitivity: {qs["sensitivity"]:4.1f}')
    fig.savefig(figure_filename, dpi=300)

In [ ]:
# get all paths for microns dataset latest published version
dandiset_id = "000037"   # OpenScope
dandiset_id = "000402"   # MICRONS

with DandiAPIClient() as client:
    assets = client.get_dandiset(dandiset_id).get_assets()
    s3_urls = [x.get_content_url(follow_redirects=1, strip_query=True) for x in assets]

In [ ]:
# create a caching scheme for DANDI downloads
cache_path = pathlib.Path('./cache');
cache_path.mkdir(parents=True, exist_ok=True)
fs = CachingFileSystem(fs=fsspec.filesystem("http"), cache_storage=str(cache_path))

In [ ]:
# make figures for file collections on DANDI
figure_path = pathlib.Path('./figures') / f"dandi-{dandiset_id}"
figure_path.mkdir(parents=True, exist_ok=True)

for url in s3_urls:
    # open the file
    with fs.open(url, "rb") as f:
        with h5py.File(f) as file:
            with pynwb.NWBHDF5IO(file=file, load_namespaces=True) as io:

                # get all two-photon series objects
                collection = (  
                    _ for _ in io.read().objects.values() 
                    if isinstance(_, pynwb.ophys.TwoPhotonSeries))

                for count, two_photon_series in enumerate(collection):
                    # dx, dy = two_photon_series.imaging_plane.grid_spacing[:]
                    # timestamps = two_photon_series.timestamps[:]
                    # for some datasets, might have to use two_photon_series.rate

                    scan = two_photon_series.data[250:750, 4:-4, 4:-4]  # remove edges because they often have digitization artifacts
                    scan = scan.transpose(0, 2, 1)
                    try:
                        make_figure(scan, figure_path / f"{url.split('/')[-1]}-{count:03}.png", 
                                title=f"NWB-id:{two_photon_series.get_ancestor().identifier}\n{two_photon_series.get_ancestor().session_id}")
                    except Exception as e:
                        print(e)